In [1]:
# Загрузка необходимых библиотек и модулей

import zipfile # Библиотека для работы с zip архивами
import os      # Библиотека для работы с фаловой системой 
import time    # Библиотека для работы со временем
import numpy as np                # Куда же без него
# import matplotlib.pyplot as plt # Для вывода изображений во время отладки

from google.colab import drive # Модуль для работы с Google Disk
from PIL import Image          # Модуль Image для работы с изображениями

In [2]:
drive.mount('/content/drive') # Подключение к диску

Mounted at /content/drive


In [3]:
# Прописываем путь к файлу с архивом
zip_file = '/content/drive/My Drive/TestTask/beeVSwasp.zip'  # Укажите здесь путь до zip-рхива в Гугл-диске (после My Drive)
                                                             # Zip-архив должен находиться в отдельном каталоге

# Распаковываем архив
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

# Просмотр результата разархивации
print(os.listdir())

['.config', 'drive', 'wasp', 'bee', 'sample_data']


In [4]:
# Читаем файлы из каталога (с измерением затраченного времени)
# В данном случае выполняется предварительная обработка изображений

img_list = []
services = ('.config', 'drive', 'sample_data')

def get_time_read():
  cur_time = time.time()          # Засекаем текущее время
  num = 0                         # Количество прочитанных файлов
  list_dir = os.listdir()         # Получаем список подкаталогов
  print(list_dir)                  
  for cur_dir in list_dir:
    if cur_dir not in services:
      list_file = os.listdir(cur_dir)
      # print(list_file)
      for cur_file in list_file:
        img = Image.open(cur_dir + '/' + cur_file) # Читаем изображение
        img = img.resize((256, 256), Image.BILINEAR)
        img = img.convert('L')
        img = (np.array(img, dtype=float) / 255).reshape(-1)
        img_list.append(img)
        num += 1
  print('Количество загруженных изображений =', num)
  print('Затрачено времени -', round(time.time() - cur_time, 2), 'cек')

In [5]:
# Считываем все картинки из базы и считаем затраченное время
get_time_read()

['.config', 'drive', 'wasp', 'bee', 'sample_data']
Количество загруженных изображений = 4596
Затрачено времени - 16.53 cек


In [6]:
img_list[2127] # Для теста. 2127 - первое изображение пчелы.

array([0.57647059, 0.57647059, 0.57254902, ..., 0.40784314, 0.4       ,
       0.39215686])

In [7]:
img_list = np.array(img_list)

In [8]:
len(img_list)

4596

In [9]:
# Создание целевых переменных в one_hot
wasp = [1,0]*2127
bee = [0,1]*2469

waspBee = wasp+bee
waspBee = np.array(waspBee).reshape(-1, 2)

waspBee.shape

(4596, 2)

In [10]:
# Функция разбиения всех данных на тестовую и проверочную выборки
def set_data(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx_train = idx[:num]
    idx_valid = idx[num:]
    
    train_data_shuffle = [data[ i] for i in idx_train]
    train_labels_shuffle = [labels[ i] for i in idx_train]
    
    valid_data_shuffle = [data[ i] for i in idx_valid]
    valid_labels_shuffle = [labels[ i] for i in idx_valid]

    return (np.array(train_data_shuffle, dtype='float32'), np.array(train_labels_shuffle),
            np.array(valid_data_shuffle, dtype='float32'), np.array(valid_labels_shuffle))

In [11]:
# Разбиение данных на тестовую и проверочную выборки в соотношении 80:20
train_size = int(0.8*len(img_list))

train_data, train_labels, valid_data, valid_labels = set_data(train_size, img_list, waspBee)

In [12]:
# Функция выбора батча
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    # print('idx', idx)
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return (np.array(data_shuffle, dtype='float32'), np.array(labels_shuffle))

In [13]:
# Установка другой версии Keras
!pip uninstall tensorflow -y
!pip install  tensorflow==1.14

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 109.3MB 1.3MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 491kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 37.3MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


Необходимо убедиться, что версия TensorFlow выбрана верно.

In [14]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [40]:
learning_rate = 0.1
epochs = 500
batch_size = 128

n_hidden_1 = 256
n_hidden_2 = 256
num_input =  65536
num_classes = 2

X = tf.compat.v1.placeholder('float', [None, num_input])
Y = tf.compat.v1.placeholder('float', [None, num_classes])

In [41]:
weights = {
    'h1' : tf.Variable(tf.random.normal([num_input, n_hidden_1])),
    'h2' : tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'output' : tf.Variable(tf.random.normal([n_hidden_2, num_classes])),    
}
biases = {
    'b1' : tf.Variable(tf.random.normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random.normal([n_hidden_2])),
    'output' : tf.Variable(tf.random.normal([num_classes])),    
}

In [42]:
def network(x):
  layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
  layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
  output_layer = tf.matmul(layer_2, weights['output']) + biases['output']
  return output_layer

In [43]:
logits = network(X)

loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y
    )
)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)

In [44]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [45]:
cur_time = time.time()

with tf.Session() as sess:
  sess.run(init)

  for epoch in range (epochs):
    batch_x, batch_y = next_batch(batch_size, train_data, train_labels)
    sess.run(train, feed_dict={X: batch_x, Y: batch_y})

    if epoch % 100 == 0:
      train_accurasy = sess.run(
          accuracy,
          feed_dict={
              X: train_data,
              Y: train_labels,
          }
      )

      print('Epoch #{}: train accuracy = {}. Затрачено времени - {} сек'.format(epoch, train_accurasy, round(time.time() - cur_time, 2)))
      cur_time = time.time()

  print('Final accuracy = {}'.format(
      sess.run(
          accuracy,
          feed_dict={
              X: valid_data,
              Y: valid_labels,
          }
      )
  ))


  cur_time = time.time()
print('Затрачено времени на одно предсказание -', round(time.time() - cur_time, 2), 'cек')

Epoch #0: train accuracy = 0.535908579826355. Затрачено времени - 4.57 сек
Epoch #100: train accuracy = 0.4662676751613617. Затрачено времени - 24.2 сек
Epoch #200: train accuracy = 0.5299238562583923. Затрачено времени - 24.27 сек
Epoch #300: train accuracy = 0.4986398220062256. Затрачено времени - 24.56 сек
Epoch #400: train accuracy = 0.5228509306907654. Затрачено времени - 24.27 сек
Final accuracy = 0.541304349899292
Затрачено времени на одно предсказание - 0.03 cек
